In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [15]:
start_date = '20221226'
end_date = '20230219'

'20220801' to '20220828' 
'20220829' to '20221030'
'20221031' to '20221225'
'20221226' to '20230219'

In [16]:
query = f"""

with base as (
    select 
        date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
        date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_week, 
        city_name, order_type, order_id, customer_id,captain_id,
        distance_final_distance,
        service_obj_service_name service,
        -- case 
        -- when ride_distance <= 5.0 then 'Short'
        -- when ride_distance > 5.0 then 'Long' 
        -- else 'test' end distance_tag,
        spd_fraud_flag,ride_time,ride_distance,
        amount,amount_breakup_base_fare_total,actual_price
        
    from orders.order_logs_snapshot
    where order_status = 'dropped' --and event_type = 'dropped'
        and yyyymmdd BETWEEN '{start_date}' and '{end_date}'
        and service_obj_service_name = 'Auto'
),

segment as (
select * 
from 
(
select 
    date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
    date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_wk, 
    captain_id, 
    case 
    when auto_pc_segment like 'HP_D%' then '3. HP Daily'
    when auto_pc_segment like 'HP_i%' then '4. HP Intra/Inter'

    when auto_pc_segment like 'MP_D%' then '5. MP Daily'
    when auto_pc_segment like 'MP_i%' then '6. MP Intra/Inter'

    when auto_pc_segment like 'LP_D%' then '7. LP Daily'
    when auto_pc_segment like 'LP_i%' then '8. LP Intra/Inter'
    
    when auto_pc_segment like 'UHP_D%' then '1. UHP Daily'
    when auto_pc_segment like 'UHP_i%' then '2. UHP Intra/Inter'
    
    end segment,
    row_number() over(partition by date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d'), captain_id order by yyyymmdd desc) seq_no
from datasets.captain_metrics_segments
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and auto_pc_segment is not null 
    -- and day_of_week(cast (date_parse(yyyymmdd, '%Y%m%d') as date)) = 7
    and captain_id in (select distinct captain_id from base)
 ) where seq_no = 1    
),

total_spd_ride as (

select 
    order_week,
    service,
    count(distinct order_id) total_orders,
    count(distinct case when spd_fraud_flag = true then order_id end) as total_spd_orders,
    count(distinct case when ride_distance <= 5.0 and spd_fraud_flag = true then order_id end) as short_spd_orders,
    count(distinct case when ride_distance > 5.0 and spd_fraud_flag = true then order_id end) as long_spd_orders
from base
group by 1,2
),

spd_ride as (
select 
    order_week,
    service,
    coalesce(segment,'0. New') segment,
    count(distinct base.customer_id) uniq_customer,
    count(distinct base.captain_id) uniq_captain,
    count(distinct order_id) as total_orders,
    count(distinct case when spd_fraud_flag = true then order_id end) as total_spd_orders,
    count(distinct case when spd_fraud_flag = true then base.customer_id end) spd_uniq_customer,
    count(distinct case when spd_fraud_flag = true then base.customer_id end) spd_uniq_captain,
    count(distinct case when ride_distance <= 5.0 and spd_fraud_flag = true then order_id end) as short_spd_orders,
    count(distinct case when ride_distance > 5.0 and spd_fraud_flag = true then order_id end) as long_spd_orders,
    sum( case when spd_fraud_flag = true then amount_breakup_base_fare_total end) as actual_fare,
    sum( case when spd_fraud_flag = true then actual_price end) as estimated_fare,
    sum( case when spd_fraud_flag = true then 0.2*actual_price end) as expected_commission
from base
left join segment s on order_week = order_wk and s.captain_id = base.captain_id
group by 1,2,3
)

select 
    a.order_week,
    -- a.service,
    a.segment,
    a.total_orders,
    a.total_spd_orders,
    try(a.total_spd_orders*100.00/b.total_spd_orders) "spd_orders_%",
    a.expected_commission,
    a.short_spd_orders,
    a.long_spd_orders,
    -- try(a.short_spd_orders*100.00/b.total_spd_orders) short_spd_distr,
    -- try(a.long_spd_orders*100.00/b.total_spd_orders) long_spd_distr,
    try(a.short_spd_orders*100.00/b.short_spd_orders) "short_spd_%",
    try(a.long_spd_orders*100.00/b.long_spd_orders) "long_spd_%" ,
    a.actual_fare,
    a.estimated_fare,
    a.uniq_captain,
    a.spd_uniq_captain,
    a.uniq_customer,
    a.spd_uniq_customer
    
from spd_ride a
left join total_spd_ride b on a.order_week = b.order_week and a.service = b.service

order by 1 desc,2


"""

In [17]:
df_code = pd.read_sql(query, conn)
df_code 

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2023-02-13,0. New,5074,255,2.30,2752.6,123,132,2.72,2.01,6133.0,13763.0,1861,224,4968,224
1,2023-02-13,1. UHP Daily,150060,270,2.44,2371.0,203,67,4.49,1.02,7337.0,11855.0,1492,267,109567,267
2,2023-02-13,2. UHP Intra/Inter,44162,90,0.81,839.8,71,19,1.57,0.29,2391.0,4199.0,713,89,37336,89
3,2023-02-13,3. HP Daily,338529,1287,11.62,15087.0,696,591,15.40,9.01,31036.0,75435.0,7533,1272,278747,1272
4,2023-02-13,4. HP Intra/Inter,94738,352,3.18,3798.2,218,134,4.82,2.04,8328.0,18991.0,3391,348,88378,348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,69963,297,2.58,4683.6,160,137,3.42,2.00,7378.0,23418.0,2615,296,65447,296
68,2022-12-26,5. MP Daily,566879,2879,24.97,43463.6,1372,1507,29.32,22.00,70428.0,217318.0,32937,2847,451890,2847
69,2022-12-26,6. MP Intra/Inter,223432,1403,12.17,20634.0,667,736,14.25,10.75,34016.0,103170.0,21962,1390,201373,1390
70,2022-12-26,7. LP Daily,170999,1309,11.35,21002.2,363,946,7.76,13.81,31556.0,105011.0,21118,1305,154772,1305


In [6]:
df1 = df_code
df1.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-08-22,0. New,76457,757,7.60,11026.4,472,285,9.29,5.84,21934.5,55132.0,11681,702,52027,702
1,2022-08-22,1. UHP Daily,164649,440,4.42,7670.2,330,110,6.49,2.26,14796.5,38351.0,1962,427,120087,427
2,2022-08-22,2. UHP Intra/Inter,28637,127,1.28,2310.4,105,22,2.07,0.45,3631.0,11552.0,552,126,23413,126
3,2022-08-22,3. HP Daily,356227,1294,13.00,19934.0,760,534,14.96,10.95,43432.0,99670.0,8021,1275,273284,1275
4,2022-08-22,4. HP Intra/Inter,75854,326,3.27,5585.0,210,116,4.13,2.38,10040.5,27925.0,2911,322,69254,322


In [10]:
df2 = df_code
df2.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-10-24,0. New,1239,204,1.96,2658.6,98,106,2.43,1.67,4992.0,13293.0,214,201,1233,201
1,2022-10-24,1. UHP Daily,75313,194,1.87,2175.2,129,65,3.19,1.02,5267.0,10876.0,1163,193,65339,193
2,2022-10-24,2. UHP Intra/Inter,21384,51,0.49,1329.8,37,14,0.92,0.22,1696.0,6649.0,458,51,19744,51
3,2022-10-24,3. HP Daily,265256,994,9.56,13097.4,536,458,13.27,7.20,26211.0,65487.0,6725,981,210709,981
4,2022-10-24,4. HP Intra/Inter,65422,291,2.80,4125.6,164,127,4.06,2.00,7456.0,20628.0,2604,291,60632,291


In [14]:
df3 = df_code
df3.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-12-19,0. New,1645,289,2.15,3248.8,118,171,2.34,2.03,6680.0,16244.0,260,274,1620,274
1,2022-12-19,1. UHP Daily,153023,315,2.34,5422.2,239,76,4.74,0.90,8623.0,27111.0,1625,311,109883,311
2,2022-12-19,2. UHP Intra/Inter,31453,65,0.48,1047.8,54,11,1.07,0.13,1758.0,5239.0,571,65,26134,65
3,2022-12-19,3. HP Daily,317378,1121,8.33,13383.4,611,510,12.12,6.06,27701.0,66917.0,6634,1093,254122,1093
4,2022-12-19,4. HP Intra/Inter,68939,292,2.17,3923.8,156,136,3.10,1.61,6993.0,19619.0,2432,290,64772,290


In [18]:
df4 = df_code
df4.head()

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2023-02-13,0. New,5074,255,2.30,2752.6,123,132,2.72,2.01,6133.0,13763.0,1861,224,4968,224
1,2023-02-13,1. UHP Daily,150060,270,2.44,2371.0,203,67,4.49,1.02,7337.0,11855.0,1492,267,109567,267
2,2023-02-13,2. UHP Intra/Inter,44162,90,0.81,839.8,71,19,1.57,0.29,2391.0,4199.0,713,89,37336,89
3,2023-02-13,3. HP Daily,338529,1287,11.62,15087.0,696,591,15.40,9.01,31036.0,75435.0,7533,1272,278747,1272
4,2023-02-13,4. HP Intra/Inter,94738,352,3.18,3798.2,218,134,4.82,2.04,8328.0,18991.0,3391,348,88378,348


In [19]:
frames = [df1,df2,df3,df4]
results = pd.concat(frames)

In [20]:
results

,order_week,segment,total_orders,total_spd_orders,spd_orders_%,expected_commission,short_spd_orders,long_spd_orders,short_spd_%,long_spd_%,actual_fare,estimated_fare,uniq_captain,spd_uniq_captain,uniq_customer,spd_uniq_customer
0,2022-08-22,0. New,76457,757,7.60,11026.4,472,285,9.29,5.84,21934.5,55132.0,11681,702,52027,702
1,2022-08-22,1. UHP Daily,164649,440,4.42,7670.2,330,110,6.49,2.26,14796.5,38351.0,1962,427,120087,427
2,2022-08-22,2. UHP Intra/Inter,28637,127,1.28,2310.4,105,22,2.07,0.45,3631.0,11552.0,552,126,23413,126
3,2022-08-22,3. HP Daily,356227,1294,13.00,19934.0,760,534,14.96,10.95,43432.0,99670.0,8021,1275,273284,1275
4,2022-08-22,4. HP Intra/Inter,75854,326,3.27,5585.0,210,116,4.13,2.38,10040.5,27925.0,2911,322,69254,322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,69963,297,2.58,4683.6,160,137,3.42,2.00,7378.0,23418.0,2615,296,65447,296
68,2022-12-26,5. MP Daily,566879,2879,24.97,43463.6,1372,1507,29.32,22.00,70428.0,217318.0,32937,2847,451890,2847
69,2022-12-26,6. MP Intra/Inter,223432,1403,12.17,20634.0,667,736,14.25,10.75,34016.0,103170.0,21962,1390,201373,1390
70,2022-12-26,7. LP Daily,170999,1309,11.35,21002.2,363,946,7.76,13.81,31556.0,105011.0,21118,1305,154772,1305


In [21]:
results.to_csv("spd_auto.csv", index = False)